In [3]:
import numpy as np
import os
import pandas as pd
import pprint
import csv
import keras
#path constants
train_path = '../../../iterations/data/final/train_norm'
test_path = '../../../iterations/data/final/test_norm'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')

Using TensorFlow backend.


In [2]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def cal_length(path, vehicle_type, label_df):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    single_len=0
    file_count=0
    file_len=0
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        file_len+=len(sample_df)
        file_count+=1
        if len(sample_df)>=single_len:
            single_len=len(sample_df)
    ave_file_len=file_len/file_count
    print("AVG Length: ",ave_file_len,' file count: ',file_count,' max_len: ',single_len)

In [4]:
cal_length(train_path,'ZV573',train_label_df)

AVG Length:  154.8896184416482  file count:  49193  max_len:  5355


In [41]:
file_len_weighted=round(392*0.7)
file_len_weighted

274

In [6]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def TraverseFiles(path, vehicle_type, label_df, length):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    lables=np.array([])
    file_list= []
    #file_len=file_len_weighted
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        if len(sample_df)==0:
            continue
        elif len(sample_df)!=0:
            sample_array=np.array(sample_df.iloc[:,0:11])
            if len(sample_array)<length:
                n_zeros=length-len(sample_array)
                sample_array=np.concatenate((np.zeros((n_zeros,11)),sample_array))
                #print(len(sample_array))
            elif len(sample_array)>length:
                sample_array=sample_array[0:length]
                #print(len(sample_array))
            elif len(sample_array)==length:
                sample_array=sample_array
                #print(len(sample_array))
            #np.float(sample_array)
            #sample_array=torch.from_numpy(sample_array).float()
            #file_list=torch.cat((file_list,sample_array[0:76,]), dim=1)
        file_list.append(sample_array)
        l=np.array(label_df.loc[label_df['sample_file_name'] == file])
        lables=np.append(lables,[l[:,1]])
        #belows are to show traversing progress (DO NOT CHANGE)
        counter += 1
        if counter == counts_per_percent:
            counter = 0
            percentage_completion += 1
            print('traversing files under', path, ':', percentage_completion, "%", end="\r", flush=True)

    return file_list,lables

In [7]:
data_array_train, data_labels_train = TraverseFiles(train_path,'ZV573',train_label_df,155)
data_array_test, data_labels_test = TraverseFiles(test_path,'ZV573',test_label_df,155)

In [8]:
x_train=np.array(data_array_train)
labels_train=np.array(data_labels_train,dtype=int)

In [9]:
x_test=np.array(data_array_test)
labels_test=np.array(data_labels_test,dtype=int)

In [10]:
print(x_train.shape)
print(labels_train.shape)
print(x_test.shape)
print(labels_test.shape)

(49193, 155, 11)
(49193,)
(12299, 155, 11)
(12299,)


In [11]:
keras.backend.clear_session()
lstm = keras.Sequential()
lstm.add(keras.layers.Dropout(0.2))
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.Dense(1, activation = 'sigmoid'))

lstm.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
lstm.fit(x_train,
    labels_train,
    epochs = 500,
    batch_size=200
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
49193/49193 [==============================] - 39s 795us/step - loss: 0.7247 - accuracy: 0.5126
Epoch 2/500
49193/49193 [==============================] - 40s 805us/step - loss: 0.6975 - accuracy: 0.5173
Epoch 3/500
49193/49193 [==============================] - 40s 817us/step - loss: 0.6926 - accuracy: 0.5251
Epoch 4/500
49193/49193 [==============================] - 40s 806us/step - loss: 0.6916 - accuracy: 0.5288
Epoch 5/500
49193/49193 [==============================] - 41s 824us/step - loss: 0.6880 - accuracy: 0.5374
Epoch 6/500
49193/49193 [==============================] - 41s 834us/step - loss: 0.6870 - accuracy: 0.5405
Epoch 7/500
49193/49193 [==============================] - 41s 840us/step - loss: 0.6842 - accuracy: 0.5502
Epoch 8/500
49193/49193 [==============================] - 39s 789us/step - loss: 0.6831 - accuracy: 0.5482
Epoch 9/500
49193/49193 [==============================] - 37s 753us/step - loss: 0.6822

49193/49193 [==============================] - 36s 735us/step - loss: 0.6662 - accuracy: 0.5750
Epoch 75/500
49193/49193 [==============================] - 36s 736us/step - loss: 0.6663 - accuracy: 0.5742
Epoch 76/500
49193/49193 [==============================] - 36s 732us/step - loss: 0.6650 - accuracy: 0.5794
Epoch 77/500
49193/49193 [==============================] - 36s 733us/step - loss: 0.6655 - accuracy: 0.5775
Epoch 78/500
49193/49193 [==============================] - 36s 735us/step - loss: 0.6650 - accuracy: 0.5790
Epoch 79/500
49193/49193 [==============================] - 36s 738us/step - loss: 0.6651 - accuracy: 0.5768
Epoch 80/500
49193/49193 [==============================] - 36s 732us/step - loss: 0.6627 - accuracy: 0.5807
Epoch 81/500
49193/49193 [==============================] - 36s 732us/step - loss: 0.6640 - accuracy: 0.5788
Epoch 82/500
49193/49193 [==============================] - 36s 733us/step - loss: 0.6639 - accuracy: 0.5819
Epoch 83/500
49193/49193 [======

49193/49193 [==============================] - 36s 730us/step - loss: 0.6591 - accuracy: 0.5849
Epoch 149/500
49193/49193 [==============================] - 36s 732us/step - loss: 0.6576 - accuracy: 0.5938
Epoch 150/500
49193/49193 [==============================] - 36s 731us/step - loss: 0.6587 - accuracy: 0.5860
Epoch 151/500
49193/49193 [==============================] - 36s 728us/step - loss: 0.6568 - accuracy: 0.5929
Epoch 152/500
49193/49193 [==============================] - 36s 729us/step - loss: 0.6583 - accuracy: 0.5883
Epoch 153/500
49193/49193 [==============================] - 36s 733us/step - loss: 0.6596 - accuracy: 0.5879
Epoch 154/500
49193/49193 [==============================] - 36s 733us/step - loss: 0.6588 - accuracy: 0.5905
Epoch 155/500
49193/49193 [==============================] - 36s 729us/step - loss: 0.6579 - accuracy: 0.5909
Epoch 156/500
49193/49193 [==============================] - 36s 730us/step - loss: 0.6575 - accuracy: 0.5908
Epoch 157/500
49193/4919

49193/49193 [==============================] - 37s 755us/step - loss: 0.6539 - accuracy: 0.5972
Epoch 223/500
49193/49193 [==============================] - 37s 753us/step - loss: 0.6558 - accuracy: 0.5931
Epoch 224/500
49193/49193 [==============================] - 37s 757us/step - loss: 0.6548 - accuracy: 0.5964
Epoch 225/500
49193/49193 [==============================] - 37s 755us/step - loss: 0.6545 - accuracy: 0.5970
Epoch 226/500
49193/49193 [==============================] - 37s 755us/step - loss: 0.6546 - accuracy: 0.5962
Epoch 227/500
49193/49193 [==============================] - 37s 761us/step - loss: 0.6548 - accuracy: 0.5944
Epoch 228/500
49193/49193 [==============================] - 38s 765us/step - loss: 0.6546 - accuracy: 0.5939
Epoch 229/500
49193/49193 [==============================] - 37s 754us/step - loss: 0.6542 - accuracy: 0.5951
Epoch 230/500
49193/49193 [==============================] - 37s 755us/step - loss: 0.6545 - accuracy: 0.5940
Epoch 231/500
49193/4919

49193/49193 [==============================] - 36s 729us/step - loss: 0.6536 - accuracy: 0.5953
Epoch 297/500
49193/49193 [==============================] - 36s 728us/step - loss: 0.6541 - accuracy: 0.5958
Epoch 298/500
49193/49193 [==============================] - 36s 730us/step - loss: 0.6523 - accuracy: 0.6004
Epoch 299/500
49193/49193 [==============================] - 36s 731us/step - loss: 0.6532 - accuracy: 0.5975
Epoch 300/500
49193/49193 [==============================] - 36s 727us/step - loss: 0.6537 - accuracy: 0.5976
Epoch 301/500
49193/49193 [==============================] - 36s 728us/step - loss: 0.6540 - accuracy: 0.5964
Epoch 302/500
49193/49193 [==============================] - 36s 729us/step - loss: 0.6537 - accuracy: 0.5961
Epoch 303/500
49193/49193 [==============================] - 36s 731us/step - loss: 0.6536 - accuracy: 0.5981
Epoch 304/500
49193/49193 [==============================] - 36s 732us/step - loss: 0.6531 - accuracy: 0.5975
Epoch 305/500
49193/4919

49193/49193 [==============================] - 36s 729us/step - loss: 0.6521 - accuracy: 0.5982
Epoch 371/500
49193/49193 [==============================] - 36s 729us/step - loss: 0.6520 - accuracy: 0.5989
Epoch 372/500
49193/49193 [==============================] - 36s 729us/step - loss: 0.6523 - accuracy: 0.5989
Epoch 373/500
49193/49193 [==============================] - 36s 728us/step - loss: 0.6526 - accuracy: 0.6001
Epoch 374/500
49193/49193 [==============================] - 36s 728us/step - loss: 0.6522 - accuracy: 0.5998
Epoch 375/500
49193/49193 [==============================] - 36s 730us/step - loss: 0.6531 - accuracy: 0.5970
Epoch 376/500
49193/49193 [==============================] - 36s 730us/step - loss: 0.6524 - accuracy: 0.5994
Epoch 377/500
49193/49193 [==============================] - 36s 741us/step - loss: 0.6522 - accuracy: 0.6013
Epoch 378/500
22600/49193 [============>.................] - ETA: 19s - loss: 0.6513 - accuracy: 0.6014

KeyboardInterrupt: 

In [13]:
lstm.save('lstm_model_ZV573.h5')

In [14]:
pred_=lstm.predict_classes(x_test)

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(labels_test, pred_,digits=4))
print(accuracy_score(labels_test, pred_))

              precision    recall  f1-score   support

           0     0.5422    0.8278    0.6552      6253
           1     0.6088    0.2772    0.3810      6046

   micro avg     0.5571    0.5571    0.5571     12299
   macro avg     0.5755    0.5525    0.5181     12299
weighted avg     0.5749    0.5571    0.5204     12299

0.5571184649158468
